<a href="https://colab.research.google.com/github/solong96/solong96/blob/main/0911_digital_cc_topic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# !pip install contextualized-topic-models
# !pip install pyldavis
!pip install --upgrade ipython
# !pip install --upgrade ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contextualized-topic-models 2.5.0 requires ipython==8.10.0, but you have ipython 8.15.0 which is incompatible.
contextualized-topic-models 2.5.0 requires ipywidgets==7.5.1, but you have ipywidgets 8.1.1 which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.15.0 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.0 which is incompatible.


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from itertools import count
import time
import csv
from tqdm import tqdm
import collections
import sys
import os
import openpyxl
import re
import pandas as pd
import numpy as np
import json
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.corpus.reader.panlex_lite import Meaning
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.data import load
from nltk.tag import pos_tag
nltk.download('stopwords')
nltk.download('punkt')
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
wordnet_lemmatizer=WordNetLemmatizer()
stops = stopwords.words('english')
newstop = ['et', 'al', 'us','factor','evidence','review','analysis','impact','effect','example','research','paper','result',"first","second","third","forth",'doe', 'C', 'elsevier', 'ltd', 'etc', 'was', 'wa', 'ha', 'springer','mdpi','evaluation','evaluating']
stops.extend(newstop)

path = '/content/drive/MyDrive'
os.chdir(path)

In [8]:
content = pd.read_excel('0913_multi_token_abstract_full.xlsx', index_col = 0)
content

,Publication Year,Article Title,Source Title,WoS Categories,Abstract,words,energy,industry,waste,traffic,building,land-use,green,biodiversity,water,heat-island,digital,label
0,2023,Energy Efficiency Model Construction of Buildi...,SUSTAINABILITY,Green & Sustainable Science & Technology; Envi...,We aim to create a feasible quantitative metho...,create feasible quantitative method calculate ...,energy,0.007705,0,traffic,building,0,0,0,0,0,0,"energy, traffic, building"
1,2003,Adaptation measures for climate change and the...,BUILDING RESEARCH AND INFORMATION,Construction & Building Technology,Climate change scenarios are discussed for Jap...,climate-change scenario japan clear implicatio...,0.017241,0,0,0,0,0,0,0.012281,0.034483,heat-island,0,heat-island
2,2021,How Cool Pavements and Green Roof Affect Build...,HEAT TRANSFER ENGINEERING,"Thermodynamics; Engineering, Mechanical; Mecha...",The urban heat island effect amplifies the glo...,urban heat-island amplify global warm phenomen...,0.048193,0,0,0,building,0,green,0,0.012048,heat-island,digital,"building, green, heat-island, digital"
3,2011,Essence Research on the Sustainable Developmen...,PROCEEDINGS OF 2011 INTERNATIONAL CONFERENCE O...,Management; Regional & Urban Planning,Three pillars of sustainable development are e...,pillar sustainable-development economic enviro...,0,industry,0,traffic,building,0,green,0,0,0,0,"industry, traffic, building, green"
4,2019,Towards the next generation of green building ...,SUSTAINABLE CITIES AND SOCIETY,Construction & Building Technology; Green & Su...,"At present, cities are under the threats of gl...",present city threat global warm urban heat-isl...,0.008929,0,0,0.006835,building,0,0.008929,0,0,heat-island,0,"building, heat-island"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52719,2014,Feasibility of Solid Fuel CLC Plant Investment...,12TH INTERNATIONAL CONFERENCE ON GREENHOUSE GA...,"Chemistry, Applied; Energy & Fuels",It is predicted that share of solar and wind e...,share solar wind energy increase future energy...,energy,industry,0,0.012545,0.004631,0,0.007018,0,0,0,0,"energy, industry"
52720,2018,Investigation of the Influence of Energy Param...,OBRABOTKA METALLOV-METAL WORKING AND MATERIAL ...,Metallurgy & Metallurgical Engineering,Justification: High-strength low-alloy steels ...,justification high-strength low-alloy steel us...,0.021505,0.005376,0,0.017724,0.016502,0,0,0,0.002913,0,0,None
52721,2005,The Mekong - Applications of Value at Risk (Va...,MODSIM 2005: INTERNATIONAL CONGRESS ON MODELLI...,"Computer Science, Interdisciplinary Applicatio...",The world's tenth largest river in terms of an...,world tenth largest river term annual flow vol...,0.013937,0,0,0,0.005347,0,0,0,0.037879,0,digital,digital
52722,2020,Water in the dynamics of strategic natural res...,RELACIONES INTERNACIONALES-MADRID,International Relations,Are we close to facing what some researchers a...,close face researcher politician call war wate...,0.015385,0.002364,0,0.005047,0,0,0,0,0.048352,0,0,None


In [11]:
category = content.columns[6:17]
print(category)

original = [[] for _ in range(len(category))]
sentence = [[] for _ in range(len(category))]
for j in tqdm(range(len(category))):
    for i in range(len(content)):
        if content[category[j]][i] == category[j]:
            original[j].append(content['Abstract'][i])
            sentence[j].append(content['words'][i])

Index(['energy', 'industry', 'waste', 'traffic', 'building', 'land-use',
       'green', 'biodiversity', 'water', 'heat-island', 'digital'],
      dtype='object')


100%|██████████| 11/11 [00:08<00:00,  1.26it/s]


In [14]:
for i in range(len(content.columns[6:17])):
    print(category[i], len(original[i]), len(sentence[i]))

energy 12796 12796
industry 7891 7891
waste 2408 2408
traffic 9685 9685
building 6022 6022
land-use 1323 1323
green 4726 4726
biodiversity 2750 2750
water 10603 10603
heat-island 489 489
digital 8135 8135


In [15]:
print(original[0][1])
print(sentence[0][1])

The low-carbon building has been proposed to mitigate the climate change caused by environmental problems and realize carbon neutrality in urban areas. In addition, the integrated energy system (IES) has been developed to reduce renewable energy curtailment in the power distribution system and improve energy efficiency due to the independent operation of traditional energy systems. In this paper, we propose a stochastic planning method for low-carbon building IES, in which the Vehicle to Grid (V2G) is also considered to further increase the flexibility of low-carbon buildings. The proposed planning method optimizes the investment and operation costs, and CO2 emission of the building IES, to achieve the maximum benefit of the low-carbon building and help realize carbon neutrality. By considering the uncertainty of distributed renewable energy, multi-energy load fluctuation and the random behavior of EV users, a two-stage stochastic programming model is formulated with chance constraints

In [16]:
for i in tqdm(range(len(category))):
    pd.DataFrame(sentence[i]).to_csv("sentence/"+str(category[i])+".txt", index=False, header=None, sep="\t")

100%|██████████| 11/11 [00:05<00:00,  1.96it/s]


In [ ]:
folder_path = "F:/02. 유영은(박사과정)/OneDrive - 충북대학교/05. 수업/08. AI 교육/ICT/digital_cc/0906/sentence/"
file_list = os.listdir(folder_path)
print(len(file_list))
for topic_name in file_list:
    print(topic_name[0:len(topic_name)-4])

11
biodiversity
building
digital
energy
green
heat-island
industry
land-use
traffic
waste
water


In [ ]:
corpus = tp.utils.Corpus()
for topic_name in file_list:
    for line in tqdm(open(folder_path + topic_name)):
        tokens = word_tokenize(line)
        corpus.add_doc(words = tokens)

        model_results = {'Topics': [], 'Opti_Topics': [], 'Coherence': [], 'Perplexity' : []}

        mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=30, corpus=corpus, seed=99999)

        # Initiate MCMC burn-in
        mdl.burn_in = 100
        mdl.train(0)

        print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
        print('Removed top words:', mdl.removed_top_words)
        print('Training...', file=sys.stderr, flush=True)

        # Train model
        mcmc_iter = 1000
        quiet = False

        step=round(mcmc_iter*0.01)
        for i in range(0, mcmc_iter, step):
            mdl.train(step, workers=1)
            if not quiet:
                ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

        coh = tp.coherence.Coherence(mdl, coherence='c_v')
        average_coherence = coh.get_score()
        topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

        print('Topic{}_Opti topic{}_Coherence: {:.4}_Perplexity: {:.4}'.format(topic_name[0:len(topic_name)-4], topic_len, average_coherence, mdl.perplexity))
        model_results['Topics'].append(k)
        model_results['Opti_Topics'].append(topic_len)
        model_results['Coherence'].append(average_coherence)
        model_results['Perplexity'].append(mdl.perplexity)

    pd.DataFrame(model_results).to_excel('0913_num_40_{}.xlsx'.format(topic_name[0:len(topic_name)-4]), index=False)

: 

: 

In [ ]:
corpus = tp.utils.Corpus()
for topic_name in file_list:
    for line in tqdm(open(folder_path + topic_name)):
        tokens = word_tokenize(line)
        corpus.add_doc(words = tokens)

    model_results = {'Topics': [], 'Opti_Topics': [], 'Coherence': [], 'Perplexity' : []}

    for k in tqdm(range(2,41)):
        mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=k, corpus=corpus, seed=99999)

        # Initiate MCMC burn-in
        mdl.burn_in = 100
        mdl.train(0)

        print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
        print('Removed top words:', mdl.removed_top_words)
        print('Training...', file=sys.stderr, flush=True)

        # Train model
        mcmc_iter = 1000
        quiet = False

        step=round(mcmc_iter*0.01)
        for i in range(0, mcmc_iter, step):
            mdl.train(step, workers=1)
            if not quiet:
                ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

        coh = tp.coherence.Coherence(mdl, coherence='c_v')
        average_coherence = coh.get_score()
        topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

        print('Topic{}_Opti topic{}_Coherence: {:.4}_Perplexity: {:.4}'.format(k, topic_len, average_coherence, mdl.perplexity))
        model_results['Topics'].append(k)
        model_results['Opti_Topics'].append(topic_len)
        model_results['Coherence'].append(average_coherence)
        model_results['Perplexity'].append(mdl.perplexity)

        # topic_results = {'Topics': [], 'document_count': [], 'represent': [], 'words': [], 'word_probability': []}

        # important_topics = [j for j, v in sorted(enumerate(mdl.get_count_by_topics()), key=lambda x:x[1], reverse=True)]
        # important_topics.sort()

        # # 먼저 토픽 개수만큼 공간을 만들어줍니다.
        # buckets = [[] for _ in range(len(important_topics))]

        # # extract candidates for auto topic labeling
        # extractor = tp.label.PMIExtractor(min_cf=5, min_df=10, min_len=1, max_len=4, max_cand=10000)
        # cands = extractor.extract(mdl)

        # labeler = tp.label.FoRelevance(mdl, cands, min_df=10, smoothing=1e-2, mu=0.25)

        # for t in tqdm(important_topics):
        #     if not mdl.is_live_topic(t): continue # remove un-assigned topics at the end (i.e. not alive)

        #     # 전체 문헌에 대해, top 1 주제 번호를 찾아 그 그룹에 해당 문헌을 포함시킵니다.
        #     for d in mdl.docs:
        #         topic = d.get_topics(top_n=1)[0][0]
        #         if t == topic:
        #             buckets[t].append(topic)

        #     topic_results['Topics'].append(t)
        #     topic_results['document_count'].append(len(buckets[t]))
        #     label = "Labels:", ', '.join(label for label, score in labeler.get_topic_labels(t, top_n=40))
        #     topic_results['represent'].append(label)
        #     word = "Words:", ', '.join(word for word, prob in mdl.get_topic_words(t, top_n=40))
        #     prob = [np.round(prob,6) for word, prob in mdl.get_topic_words(t, top_n=40)]
        #     topic_results['words'].append(word)
        #     word_porobability = dict(zip(word, prob))
        #     topic_results['word_probability'].append(word_porobability)

        # pd.DataFrame(topic_results).to_excel(writer, sheet_name = 'Topics_{}'.format(t), index=False)

    pd.DataFrame(model_results).to_excel('0913_num_40_{}.xlsx'.format(topic_name[0:len(topic_name)-4]), index=False)

2750it [00:00, 15003.72it/s]
  0%|          | 0/39 [00:00<?, ?it/s]

: 

: 

In [ ]:
corpus = tp.utils.Corpus()
for line in tqdm(open('0527_token_cc.txt')):
    tokens = word_tokenize(line)
    corpus.add_doc(words = tokens)

mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=model_results['Coherence'].index(max(model_results['Coherence']))+2, corpus=corpus, seed=99999)

# Initiate MCMC burn-in
mdl.burn_in = 100
mdl.train(0)

print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
print('Training...', file=sys.stderr, flush=True)

# Train model
mcmc_iter = 1000
quiet = False

step=round(mcmc_iter*0.01)
for i in range(0, mcmc_iter, step):
    mdl.train(step, workers=1)
    if not quiet:
        ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

coh = tp.coherence.Coherence(mdl, coherence='c_v')
average_coherence = coh.get_score()
topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

print('topic{}_opti topic{}_Coherence: {:.4}_Perplexity: {:.4}'.format(38, topic_len, average_coherence, mdl.perplexity))

topic_results = {'Topics': [], 'document_count': [], 'represent': [], 'words': [], 'word_probability': []}

important_topics = [k for k, v in sorted(enumerate(mdl.get_count_by_topics()), key=lambda x:x[1], reverse=True)]
important_topics.sort()

# 먼저 토픽 개수만큼 공간을 만들어줍니다.
buckets = [[] for _ in range(len(important_topics))]

# extract candidates for auto topic labeling
extractor = tp.label.PMIExtractor(min_cf=5, min_df=10, min_len=1, max_len=4, max_cand=10000)
cands = extractor.extract(mdl)

labeler = tp.label.FoRelevance(mdl, cands, min_df=10, smoothing=1e-2, mu=0.25)

for k in tqdm(important_topics):
    if not mdl.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)

    # 전체 문헌에 대해, top 1 주제 번호를 찾아 그 그룹에 해당 문헌을 포함시킵니다.
    for d in mdl.docs:
        topic = d.get_topics(top_n=1)[0][0]
        if k == topic:
            buckets[k].append(topic)

    topic_results['Topics'].append(k)
    topic_results['document_count'].append(len(buckets[k]))
    label = "Labels:", ', '.join(label for label, score in labeler.get_topic_labels(k, top_n=40))
    topic_results['represent'].append(label)
    word = "Words:", ', '.join(word for word, prob in mdl.get_topic_words(k, top_n=40))
    prob = [np.round(prob,6) for word, prob in mdl.get_topic_words(k, top_n=40)]
    topic_results['words'].append(word)
    word_porobability = dict(zip(word, prob))
    topic_results['word_probability'].append(word_porobability)

pd.DataFrame(topic_results).to_excel('0527_result_topic_cc.xlsx', index=False)

33131it [00:12, 2723.48it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\2904594182.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


Num docs: 33131 , Vocab size: 10247 , Num words: 2857958
Removed top words: ['energy', 'system', 'climate-change', 'emission', 'carbon', 'policy', 'study', 'water', 'model', 'development']
topic38_opti topic40_Coherence: 0.7693_Perplexity: 2.075e+03


100%|██████████| 40/40 [00:02<00:00, 14.91it/s]


In [ ]:
topic_doc = {'mdl.docs': [],
             'topic': []}


# 먼저 토픽 개수만큼 공간을 만들어줍니다.
buckets = [[] for _ in range(mdl.k)]
# 전체 문헌에 대해, top 1 주제 번호를 찾아 그 그룹에 해당 문헌을 포함시킵니다.
for d in mdl.docs:
    buckets[d.get_topics(top_n=1)[0][0]].append(d)

for k in range(mdl.k):
    print('Num docs belonging to topic #%d: %d' % (k, len(buckets[k])))

topic = [[] for _ in range(mdl.k)]
for k in range(mdl.k):
    for d in buckets[k]:
        topic_doc['mdl.docs'].append(d)
        topic_doc['topic'].append(k)

pd.DataFrame(topic_doc).to_excel('0527_topic_doc_cc.xlsx', index=False)
topic_doc = pd.DataFrame(topic_doc)
topic_doc

Num docs belonging to topic #0: 5046
Num docs belonging to topic #1: 736
Num docs belonging to topic #2: 380
Num docs belonging to topic #3: 31
Num docs belonging to topic #4: 7
Num docs belonging to topic #5: 416
Num docs belonging to topic #6: 2640
Num docs belonging to topic #7: 561
Num docs belonging to topic #8: 9
Num docs belonging to topic #9: 426
Num docs belonging to topic #10: 66
Num docs belonging to topic #11: 6
Num docs belonging to topic #12: 1297
Num docs belonging to topic #13: 6
Num docs belonging to topic #14: 54
Num docs belonging to topic #15: 271
Num docs belonging to topic #16: 238
Num docs belonging to topic #17: 31
Num docs belonging to topic #18: 2491
Num docs belonging to topic #19: 1921
Num docs belonging to topic #20: 234
Num docs belonging to topic #21: 8
Num docs belonging to topic #22: 503
Num docs belonging to topic #23: 1421
Num docs belonging to topic #24: 1170
Num docs belonging to topic #25: 937
Num docs belonging to topic #26: 2099
Num docs belongin

,mdl.docs,topic
0,"(climate-change, global, threat, time, signato...",0
1,"(multiple, vision, urbanism, respond, climate,...",0
2,"(valuable, work, climate, crisis, contribution...",0
3,"(article, stock, climate-change, special, IPCC...",0
4,"(affected, climate-change, main, challenge, ag...",0
...,...,...
33126,"(compare, energy, ventilation, performance, le...",35
33127,"(letter, editor)",37
33128,"(solar, system, water, water, earth, water, co...",37
33129,"(earth, water, core, sustainable-development, ...",37


In [ ]:
full_content = {'No' : [], 'corpus' : [], 'Article Title' : [], 'Abstract' : [], 'Year' : []}

full_content['No'] = range(0,len(corpus))
full_content['corpus'] =  corpus
full_content['Article Title'] = content['Article Title']
full_content['Abstract'] = content['Abstract']
# full_content['sentence'] = sentence
full_content['Year'] = content['Publication Year']

df = pd.DataFrame(full_content)
df.to_excel('0527_full_content_cc.xlsx')
df

,No,corpus,Article Title,Abstract,Year
0,0,"(climate-change, global, threat, time, signato...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019
1,1,"(multiple, vision, urbanism, respond, climate,...",Transnational Governance and the Urban Politic...,Multiple visions for how urbanism can respond ...,2022
2,2,"(climate-change, scenario, japan, clear, impli...",Adaptation measures for climate change and the...,Climate change scenarios are discussed for Jap...,2003
3,3,"(valuable, work, climate, crisis, contribution...",Addressing the Climate Crisis: An Action Plan ...,Psychologists have conducted valuable work on ...,2022
4,4,"(article, stock, climate-change, special, IPCC...","Global warming, state of scientific knowledge,...","This article takes stock of climate change, ba...",2020
...,...,...,...,...,...
33126,33126,"(purposethis, study, aim, describe, bibliometr...",A systematic bibliometric analysis of studies ...,PurposeThis study aims to describe a bibliomet...,2023
33127,33127,"(increase, price, dependence, fossil, fuel, co...",Dual fuel ethanol port injection in a compress...,"The increase in prices, dependence on fossil f...",2021
33128,33128,"(streptomyces, specie, antibiotic, production,...",A Proteomic Analysis Indicates That Oxidative ...,"In most Streptomyces species, antibiotic produ...",2022
33129,33129,"(synthesis, procedure, dodecanethiol, wurtzite...",Understanding quantum confinement and ligand r...,We report a synthesis procedure for dodecaneth...,2018


In [ ]:
full_docs = pd.read_excel('0527_full_content_cc.xlsx',dtype=object)
topic_docs = pd.read_excel('0527_topic_doc_cc.xlsx',dtype=object)

In [ ]:
full_docs = full_docs.drop(['Unnamed: 0'], axis=1)
full_docs

,No,corpus,Article Title,Abstract,Year
0,0,"<tomotopy.Document with words=""climate-change ...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019
1,1,"<tomotopy.Document with words=""multiple vision...",Transnational Governance and the Urban Politic...,Multiple visions for how urbanism can respond ...,2022
2,2,"<tomotopy.Document with words=""climate-change ...",Adaptation measures for climate change and the...,Climate change scenarios are discussed for Jap...,2003
3,3,"<tomotopy.Document with words=""valuable work c...",Addressing the Climate Crisis: An Action Plan ...,Psychologists have conducted valuable work on ...,2022
4,4,"<tomotopy.Document with words=""article stock c...","Global warming, state of scientific knowledge,...","This article takes stock of climate change, ba...",2020
...,...,...,...,...,...
33126,33126,"<tomotopy.Document with words=""purposethis stu...",A systematic bibliometric analysis of studies ...,PurposeThis study aims to describe a bibliomet...,2023
33127,33127,"<tomotopy.Document with words=""increase price ...",Dual fuel ethanol port injection in a compress...,"The increase in prices, dependence on fossil f...",2021
33128,33128,"<tomotopy.Document with words=""streptomyces sp...",A Proteomic Analysis Indicates That Oxidative ...,"In most Streptomyces species, antibiotic produ...",2022
33129,33129,"<tomotopy.Document with words=""synthesis proce...",Understanding quantum confinement and ligand r...,We report a synthesis procedure for dodecaneth...,2018


In [ ]:
topic_docs

,mdl.docs,topic
0,"<tomotopy.Document with words=""climate-change ...",0
1,"<tomotopy.Document with words=""multiple vision...",0
2,"<tomotopy.Document with words=""valuable work c...",0
3,"<tomotopy.Document with words=""article stock c...",0
4,"<tomotopy.Document with words=""affected climat...",0
...,...,...
33126,"<tomotopy.Document with words=""compare energy ...",35
33127,"<tomotopy.Document with words=""letter editor"">",37
33128,"<tomotopy.Document with words=""solar system wa...",37
33129,"<tomotopy.Document with words=""earth water cor...",37


In [ ]:
full_docs2 = full_docs[['No','corpus','Article Title','Abstract','Year']]
df = pd.merge(full_docs2, topic_docs, how = 'left', left_on = 'corpus', right_on = 'mdl.docs')
df

,No,corpus,Article Title,Abstract,Year,mdl.docs,topic
0,0,"<tomotopy.Document with words=""climate-change ...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019,"<tomotopy.Document with words=""climate-change ...",0
1,1,"<tomotopy.Document with words=""multiple vision...",Transnational Governance and the Urban Politic...,Multiple visions for how urbanism can respond ...,2022,"<tomotopy.Document with words=""multiple vision...",0
2,2,"<tomotopy.Document with words=""climate-change ...",Adaptation measures for climate change and the...,Climate change scenarios are discussed for Jap...,2003,"<tomotopy.Document with words=""climate-change ...",5
3,3,"<tomotopy.Document with words=""valuable work c...",Addressing the Climate Crisis: An Action Plan ...,Psychologists have conducted valuable work on ...,2022,"<tomotopy.Document with words=""valuable work c...",0
4,4,"<tomotopy.Document with words=""article stock c...","Global warming, state of scientific knowledge,...","This article takes stock of climate change, ba...",2020,"<tomotopy.Document with words=""article stock c...",0
...,...,...,...,...,...,...,...
33146,33126,"<tomotopy.Document with words=""purposethis stu...",A systematic bibliometric analysis of studies ...,PurposeThis study aims to describe a bibliomet...,2023,"<tomotopy.Document with words=""purposethis stu...",30
33147,33127,"<tomotopy.Document with words=""increase price ...",Dual fuel ethanol port injection in a compress...,"The increase in prices, dependence on fossil f...",2021,"<tomotopy.Document with words=""increase price ...",17
33148,33128,"<tomotopy.Document with words=""streptomyces sp...",A Proteomic Analysis Indicates That Oxidative ...,"In most Streptomyces species, antibiotic produ...",2022,"<tomotopy.Document with words=""streptomyces sp...",1
33149,33129,"<tomotopy.Document with words=""synthesis proce...",Understanding quantum confinement and ligand r...,We report a synthesis procedure for dodecaneth...,2018,"<tomotopy.Document with words=""synthesis proce...",7


In [ ]:
print(df.duplicated(['Abstract']).sum())
df2 = df.drop_duplicates(['Abstract'],keep='first',ignore_index=True)
df2

20


,No,corpus,Article Title,Abstract,Year,mdl.docs,topic
0,0,"<tomotopy.Document with words=""climate-change ...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019,"<tomotopy.Document with words=""climate-change ...",0
1,1,"<tomotopy.Document with words=""multiple vision...",Transnational Governance and the Urban Politic...,Multiple visions for how urbanism can respond ...,2022,"<tomotopy.Document with words=""multiple vision...",0
2,2,"<tomotopy.Document with words=""climate-change ...",Adaptation measures for climate change and the...,Climate change scenarios are discussed for Jap...,2003,"<tomotopy.Document with words=""climate-change ...",5
3,3,"<tomotopy.Document with words=""valuable work c...",Addressing the Climate Crisis: An Action Plan ...,Psychologists have conducted valuable work on ...,2022,"<tomotopy.Document with words=""valuable work c...",0
4,4,"<tomotopy.Document with words=""article stock c...","Global warming, state of scientific knowledge,...","This article takes stock of climate change, ba...",2020,"<tomotopy.Document with words=""article stock c...",0
...,...,...,...,...,...,...,...
33126,33126,"<tomotopy.Document with words=""purposethis stu...",A systematic bibliometric analysis of studies ...,PurposeThis study aims to describe a bibliomet...,2023,"<tomotopy.Document with words=""purposethis stu...",30
33127,33127,"<tomotopy.Document with words=""increase price ...",Dual fuel ethanol port injection in a compress...,"The increase in prices, dependence on fossil f...",2021,"<tomotopy.Document with words=""increase price ...",17
33128,33128,"<tomotopy.Document with words=""streptomyces sp...",A Proteomic Analysis Indicates That Oxidative ...,"In most Streptomyces species, antibiotic produ...",2022,"<tomotopy.Document with words=""streptomyces sp...",1
33129,33129,"<tomotopy.Document with words=""synthesis proce...",Understanding quantum confinement and ligand r...,We report a synthesis procedure for dodecaneth...,2018,"<tomotopy.Document with words=""synthesis proce...",7


In [ ]:
df2['count'] = 1
df2

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\1386363151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['count'] = 1


,No,corpus,Article Title,Abstract,Year,mdl.docs,topic,count
0,0,"<tomotopy.Document with words=""climate-change ...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019,"<tomotopy.Document with words=""climate-change ...",0,1
1,1,"<tomotopy.Document with words=""multiple vision...",Transnational Governance and the Urban Politic...,Multiple visions for how urbanism can respond ...,2022,"<tomotopy.Document with words=""multiple vision...",0,1
2,2,"<tomotopy.Document with words=""climate-change ...",Adaptation measures for climate change and the...,Climate change scenarios are discussed for Jap...,2003,"<tomotopy.Document with words=""climate-change ...",5,1
3,3,"<tomotopy.Document with words=""valuable work c...",Addressing the Climate Crisis: An Action Plan ...,Psychologists have conducted valuable work on ...,2022,"<tomotopy.Document with words=""valuable work c...",0,1
4,4,"<tomotopy.Document with words=""article stock c...","Global warming, state of scientific knowledge,...","This article takes stock of climate change, ba...",2020,"<tomotopy.Document with words=""article stock c...",0,1
...,...,...,...,...,...,...,...,...
33126,33126,"<tomotopy.Document with words=""purposethis stu...",A systematic bibliometric analysis of studies ...,PurposeThis study aims to describe a bibliomet...,2023,"<tomotopy.Document with words=""purposethis stu...",30,1
33127,33127,"<tomotopy.Document with words=""increase price ...",Dual fuel ethanol port injection in a compress...,"The increase in prices, dependence on fossil f...",2021,"<tomotopy.Document with words=""increase price ...",17,1
33128,33128,"<tomotopy.Document with words=""streptomyces sp...",A Proteomic Analysis Indicates That Oxidative ...,"In most Streptomyces species, antibiotic produ...",2022,"<tomotopy.Document with words=""streptomyces sp...",1,1
33129,33129,"<tomotopy.Document with words=""synthesis proce...",Understanding quantum confinement and ligand r...,We report a synthesis procedure for dodecaneth...,2018,"<tomotopy.Document with words=""synthesis proce...",7,1


In [ ]:
df2['doc_count'] = df2.groupby(['topic'])['count'].transform(sum)
df2 = df2.sort_values('doc_count', ascending=False)
df2.to_excel('0527_final_topic_index_cc.xlsx', index=False)
df2

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\2407136449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['doc_count'] = df2.groupby(['topic'])['count'].transform(sum)


,No,corpus,Article Title,Abstract,Year,mdl.docs,topic,count,doc_count
0,0,"<tomotopy.Document with words=""climate-change ...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019,"<tomotopy.Document with words=""climate-change ...",0,1,5046
9142,9142,"<tomotopy.Document with words=""heritage practi...",Limiting binary thinking: architectural design...,Heritage practitioners are increasingly facing...,2020,"<tomotopy.Document with words=""heritage practi...",0,1,5046
9218,9218,"<tomotopy.Document with words=""acceleration te...",Are vine-growers climate change skeptics? The ...,Researchers have emphasized an acceleration of...,2020,"<tomotopy.Document with words=""acceleration te...",0,1,5046
9215,9215,"<tomotopy.Document with words=""quantitative de...",Measuring the Co-Benefits of Climate Change Mi...,Co-benefits rarely enter quantitative decision...,2014,"<tomotopy.Document with words=""quantitative de...",0,1,5046
9213,9213,"<tomotopy.Document with words=""objective artic...","Climate Change, Aeroallergens, Natural Particu...",The objective of this article is to systematic...,2011,"<tomotopy.Document with words=""objective artic...",0,1,5046
...,...,...,...,...,...,...,...,...,...
19344,19344,"<tomotopy.Document with words=""earth water cor...",Long waves in oceanic shallow water: Symbolic ...,"For the Earth, water is at the core of sustain...",2020,"<tomotopy.Document with words=""earth water cor...",37,1,3
18984,18984,"<tomotopy.Document with words=""solar system wa...","Water-wave symbolic computation for the Earth,...","In the Solar System, water and water waves are...",2020,"<tomotopy.Document with words=""solar system wa...",37,1,3
8409,8409,"<tomotopy.Document with words=""letter editor"">",Formally designate blue-green infrastructure f...,Letter to the Editor,2022,"<tomotopy.Document with words=""letter editor"">",37,1,3
26573,26573,"<tomotopy.Document with words=""GAINS global ai...",Progress and recent developments in the GAINS ...,The GAINS (Global Air-ocean IN-situ System) ne...,2001,"<tomotopy.Document with words=""GAINS global ai...",38,1,1


In [ ]:
content = df2
content = content[content['doc_count']>=10]
content = content.reset_index(drop=True)
content = content.drop(['No', 'mdl.docs'], axis=1)
content

,corpus,Article Title,Abstract,Year,topic,count,doc_count
0,"<tomotopy.Document with words=""climate-change ...",Aligning Urban Policy with Climate Action in t...,Climate change is the biggest global threat of...,2019,0,1,5046
1,"<tomotopy.Document with words=""heritage practi...",Limiting binary thinking: architectural design...,Heritage practitioners are increasingly facing...,2020,0,1,5046
2,"<tomotopy.Document with words=""acceleration te...",Are vine-growers climate change skeptics? The ...,Researchers have emphasized an acceleration of...,2020,0,1,5046
3,"<tomotopy.Document with words=""quantitative de...",Measuring the Co-Benefits of Climate Change Mi...,Co-benefits rarely enter quantitative decision...,2014,0,1,5046
4,"<tomotopy.Document with words=""objective artic...","Climate Change, Aeroallergens, Natural Particu...",The objective of this article is to systematic...,2011,0,1,5046
...,...,...,...,...,...,...,...
33070,"<tomotopy.Document with words=""focus uncertain...",Climate Policy and Industry Elite Perceptions ...,There has been a growing focus on uncertainty ...,2020,32,1,12
33071,"<tomotopy.Document with words=""climate action ...",Social involvement on urban climate action and...,Climate action has been consolidated in cities...,2023,32,1,12
33072,"<tomotopy.Document with words=""continuous deve...",Design of a wave energy converter based on Hal...,With the continuous development of society and...,2020,32,1,12
33073,"<tomotopy.Document with words=""MNP-23 septum-l...",Septum magnet MNP-23 for the CERN PS experimen...,Two MNP-23 septum-like magnets are installed a...,2004,32,1,12


In [ ]:
topics = list(content['topic'].unique())
topics

[0,
 31,
 33,
 30,
 6,
 18,
 26,
 19,
 23,
 12,
 24,
 25,
 1,
 7,
 22,
 9,
 5,
 2,
 15,
 16,
 20,
 29,
 10,
 14,
 17,
 3,
 32]

In [ ]:
print(len(content.Abstract.values[content['topic'] == topics[0]]))
print(content.Abstract.values[content['topic'] == topics[0]][0])
print(content.Abstract.values[content['topic'] == topics[0]])

5046
Climate change is the biggest global threat of our time. As a signatory nation of the Paris Agreement, Brazil has made a climate action commitment, and expressed its nationally determined contribution to reduce its greenhouse gas emissions by 37%. The Brazilian population is highly urban, and Brazilian cities are mostly responsible for greenhouse gas emissions, and the worst effects of global warming are experienced in cities. Hence, the fulfillment of the Brazilian climate commitments depends on the active engagement of municipalities. Nevertheless, the Brazilian national government does not monitor local climate actions, and it is not clear how local urban policy is aligned with climate action. In order to bridge this gap, this study tackles the question of: Are, and if yes how, cities considering the climate emergency in their local planning mechanisms? This question is investigated by placing five major Brazilian cities under the microscope. The methodological approach include

In [ ]:
custom_tokenizer = CustomTokenizer()
sentence = []
for i in topics:
    preprocessing = []
    for k in tqdm(range(len(content.Abstract.values[content['topic'] == i]))):
        preprocessing.append(custom_tokenizer(content.Abstract.values[content['topic'] == i][k]))
    sentence.append(preprocessing)


100%|██████████| 12/12 [00:00<00:00, 93.02it/s] 


In [ ]:
print(content.Abstract.values[content['topic'] == topics[0]][0])
print(sentence[0][0])

print(content.Abstract.values[content['topic'] == topics[0]][3000])
print(sentence[0][3000])

Climate change is the biggest global threat of our time. As a signatory nation of the Paris Agreement, Brazil has made a climate action commitment, and expressed its nationally determined contribution to reduce its greenhouse gas emissions by 37%. The Brazilian population is highly urban, and Brazilian cities are mostly responsible for greenhouse gas emissions, and the worst effects of global warming are experienced in cities. Hence, the fulfillment of the Brazilian climate commitments depends on the active engagement of municipalities. Nevertheless, the Brazilian national government does not monitor local climate actions, and it is not clear how local urban policy is aligned with climate action. In order to bridge this gap, this study tackles the question of: Are, and if yes how, cities considering the climate emergency in their local planning mechanisms? This question is investigated by placing five major Brazilian cities under the microscope. The methodological approach includes lit

In [ ]:
for t in range(len(topics)):
    print(t, len(sentence[t]))

0 5046
1 4500
2 2767
3 2706
4 2640
5 2491
6 2099
7 1921
8 1421
9 1297
10 1170
11 937
12 736
13 561
14 503
15 426
16 416
17 380
18 271
19 238
20 234
21 121
22 66
23 54
24 31
25 31
26 12


In [ ]:
sub = {'Topics' : [], 'Sub_topics' : [], 'Coherence_score' : []}
for t in range(len(topics)):
    corpus = tp.utils.Corpus()
    for line in sentence[t]:
        tokens = word_tokenize(line)
        corpus.add_doc(words = tokens)

    model_results = {'Topics': [], 'Coherence': [], 'Perplexity' : []}

    for k in tqdm(range(2,11)):
        mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=k, corpus=corpus, seed=99999)

        # Initiate MCMC burn-in
        mdl.burn_in = 100
        mdl.train(0)

        print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
        print('Removed top words:', mdl.removed_top_words)
        print('Training...', file=sys.stderr, flush=True)

        # Train model
        mcmc_iter = 1000
        quiet = False

        step=round(mcmc_iter*0.01)
        for i in range(0, mcmc_iter, step):
            mdl.train(step, workers=1)
            if not quiet:
                ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

        coh = tp.coherence.Coherence(mdl, coherence='c_v')
        average_coherence = coh.get_score()
        topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

        print('topic{}_Coherence: {:.4}_Perplexity: {:.4}'.format(topic_len, average_coherence, mdl.perplexity))
        model_results['Topics'].append(topic_len)
        model_results['Coherence'].append(average_coherence)
        model_results['Perplexity'].append(mdl.perplexity)

    pd.DataFrame(model_results).to_excel(f'sub/0527_num_10_cc_topic{t}.xlsx', index=False)

    print('Topics : {}, Sub_topics : {}, Coherence_score : {:.4}'.format(t, model_results['Coherence'].index(max(model_results['Coherence']))+2, max(model_results['Coherence'])))
    sub['Topics'].append(t)
    sub['Sub_topics'].append(model_results['Coherence'].index(max(model_results['Coherence']))+2)
    sub['Coherence_score'].append(max(model_results['Coherence']))

pd.DataFrame(sub).to_excel('sub/0527_num_10_cc_SUB_topic.xlsx', index=False)

  0%|          | 0/9 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\3125301607.py:15: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


Num docs: 5045 , Vocab size: 3230 , Num words: 396172
Removed top words: ['climate-change', 'policy', 'adaptation', 'urban', 'climate', 'system', 'city', 'local', 'development', 'water']


 11%|█         | 1/9 [00:38<05:08, 38.61s/it]Training...


topic2_Coherence: 0.3898_Perplexity: 1.856e+03
Num docs: 5045 , Vocab size: 3230 , Num words: 396172
Removed top words: ['climate-change', 'policy', 'adaptation', 'urban', 'climate', 'system', 'city', 'local', 'development', 'water']


 22%|██▏       | 2/9 [01:22<04:53, 41.89s/it]Training...


...

Num docs: 31 , Vocab size: 28 , Num words: 630
Removed top words: ['health', 'air', 'policy', 'emission', 'pollution', 'climate-change', 'scenario', 'PM2', 'energy', 'climate']
topic4_Coherence: 0.425_Perplexity: 35.81
Num docs: 31 , Vocab size: 28 , Num words: 630
Removed top words: ['health', 'air', 'policy', 'emission', 'pollution', 'climate-change', 'scenario', 'PM2', 'energy', 'climate']


100%|██████████| 9/9 [00:00<00:00,  9.95it/s]


topic2_Coherence: 0.4054_Perplexity: 35.94
Topics : 25, Sub_topics : 5, Coherence_score : 0.444


  0%|          | 0/9 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\3125301607.py:15: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
Training...
100%|██████████| 9/9 [00:00<00:00, 81.82it/s]

Num docs: 12 , Vocab size: 0 , Num words: 0
Removed top words: ['design', 'magnet', 'seal', 'energy', 'coil', 'climate', 'coffee', 'WEC', 'climate-change', 'system']
topic0_Coherence: nan_Perplexity: nan
Num docs: 12 , Vocab size: 0 , Num words: 0
Removed top words: ['design', 'magnet', 'seal', 'energy', 'coil', 'climate', 'coffee', 'WEC', 'climate-change', 'system']
topic0_Coherence: nan_Perplexity: nan
Num docs: 12 , Vocab size: 0 , Num words: 0
Removed top words: ['design', 'magnet', 'seal', 'energy', 'coil', 'climate', 'coffee', 'WEC', 'climate-change', 'system']
topic0_Coherence: nan_Perplexity: nan
Num docs: 12 , Vocab size: 0 , Num words: 0
Removed top words: ['design', 'magnet', 'seal', 'energy', 'coil', 'climate', 'coffee', 'WEC', 'climate-change', 'system']
topic0_Coherence: nan_Perplexity: nan
Num docs: 12 , Vocab size: 0 , Num words: 0
Removed top words: ['design', 'magnet', 'seal', 'energy', 'coil', 'climate', 'coffee', 'WEC', 'climate-change', 'system']
topic0_Coherence: 

In [ ]:
sub = pd.read_excel('sub/0527_num_10_cc_SUB_topic.xlsx',dtype=object)
sub = sub.dropna()
sub

,Topics,Sub_topics,Coherence_score
0,0,10,0.652704
1,1,9,0.647099
2,2,5,0.685042
3,3,10,0.655532
4,4,6,0.679814
5,5,5,0.652061
6,6,8,0.592023
7,7,10,0.606983
8,8,6,0.619624
9,9,8,0.649973


In [ ]:
writer = pd.ExcelWriter('sub/0527_results_cc_sub_topic.xlsx', engine='xlsxwriter')
writer1 = pd.ExcelWriter('sub/0527_results_cc_sub_topic_doc.xlsx', engine='xlsxwriter')

for t in sub['Topics']:
# for t in range(len(0,1)):
    corpus = tp.utils.Corpus()
    for line in sentence[t]:
        tokens = word_tokenize(line)
        corpus.add_doc(words = tokens)

    print(t, sub['Sub_topics'][t],  sub['Coherence_score'][t])

    mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=sub['Sub_topics'][t], corpus=corpus, seed=99999)

    # Initiate MCMC burn-in
    mdl.burn_in = 100
    mdl.train(0)

    print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
    print('Removed top words:', mdl.removed_top_words)
    print('Training...', file=sys.stderr, flush=True)

    # Train model
    mcmc_iter = 1000
    quiet = False

    step=round(mcmc_iter*0.01)
    for i in range(0, mcmc_iter, step):
        mdl.train(step, workers=1)
        if not quiet:
            ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

    coh = tp.coherence.Coherence(mdl, coherence='c_v')
    average_coherence = coh.get_score()
    topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

    print('topic{}_Coherence: {:.4}_Perplexity: {:.4}'.format(topic_len, average_coherence, mdl.perplexity))
    print(mdl.k, topic_len)

    topic_results = {'Topics': [], 'document_count': [], 'represent': [], 'words': [], 'word_probability': []}

    important_topics = [k for k, v in sorted(enumerate(mdl.get_count_by_topics()), key=lambda x:x[1], reverse=True)]
    important_topics.sort()

    # 먼저 토픽 개수만큼 공간을 만들어줍니다.
    buckets = [[] for _ in range(len(important_topics))]
    docs = [[] for _ in range(len(important_topics))]

    # extract candidates for auto topic labeling
    extractor = tp.label.PMIExtractor(min_cf=5, min_df=10, min_len=1, max_len=4, max_cand=10000)
    cands = extractor.extract(mdl)

    labeler = tp.label.FoRelevance(mdl, cands, min_df=10, smoothing=1e-2, mu=0.25)

    topic_doc = {'mdl.docs': [],'topic': []}

    for k in tqdm(important_topics):
        if not mdl.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)

        # 전체 문헌에 대해, top 1 주제 번호를 찾아 그 그룹에 해당 문헌을 포함시킵니다.
        for d in mdl.docs:
            topic = d.get_topics(top_n=1)[0][0]
            if k == topic:
                buckets[k].append(topic)

        topic_results['Topics'].append(k)
        topic_results['document_count'].append(len(buckets[k]))
        label = "Labels:", ', '.join(label for label, score in labeler.get_topic_labels(k, top_n=40))
        topic_results['represent'].append(label)
        word = [word for word, prob in mdl.get_topic_words(k, top_n=40)]
        prob = [np.round(prob,6) for word, prob in mdl.get_topic_words(k, top_n=40)]
        topic_results['words'].append(word)
        word_porobability = dict(zip(word, prob))
        topic_results['word_probability'].append(word_porobability)

    for d in mdl.docs:
            docs[d.get_topics(top_n=1)[0][0]].append(d)

    for k in tqdm(important_topics):
        print('Num docs belonging to topic #%d: %d' % (k, len(docs[k])))

    topic = [[] for _ in range(len(important_topics))]
    for k in tqdm(important_topics):
        for d in docs[k]:
            topic_doc['mdl.docs'].append(d)
            topic_doc['topic'].append(k)

    pd.DataFrame(topic_doc).to_excel(writer1, sheet_name = 'Topic_{}'.format(t), index=False)
    pd.DataFrame(topic_results).to_excel(writer, sheet_name = 'Topic_{}'.format(t), index=False)

writer.save()
writer1.save()

0 10 0.65270392393524


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\4137851866.py:17: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


Num docs: 5045 , Vocab size: 3230 , Num words: 396172
Removed top words: ['climate-change', 'policy', 'adaptation', 'urban', 'climate', 'system', 'city', 'local', 'development', 'water']
topic11_Coherence: 0.6527_Perplexity: 1.743e+03
11 11


100%|██████████| 11/11 [00:00<?, ?it/s]


Num docs belonging to topic #0: 8
Num docs belonging to topic #1: 4
Num docs belonging to topic #2: 10
Num docs belonging to topic #3: 5
Num docs belonging to topic #4: 6
Num docs belonging to topic #5: 400
Num docs belonging to topic #6: 22
Num docs belonging to topic #7: 2728
Num docs belonging to topic #8: 4
Num docs belonging to topic #9: 1858
Num docs belonging to topic #10: 0


...

100%|██████████| 37/37 [00:00<?, ?it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\4137851866.py:17: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


25 5 0.4439542956731242
Num docs: 31 , Vocab size: 28 , Num words: 630
Removed top words: ['health', 'air', 'policy', 'emission', 'pollution', 'climate-change', 'scenario', 'PM2', 'energy', 'climate']
topic2_Coherence: 0.444_Perplexity: 35.57
23 2


100%|██████████| 23/23 [00:00<?, ?it/s]


Num docs belonging to topic #0: 0
Num docs belonging to topic #1: 30
Num docs belonging to topic #2: 0
Num docs belonging to topic #3: 1
Num docs belonging to topic #4: 0
Num docs belonging to topic #5: 0
Num docs belonging to topic #6: 0
Num docs belonging to topic #7: 0
Num docs belonging to topic #8: 0
Num docs belonging to topic #9: 0
Num docs belonging to topic #10: 0
Num docs belonging to topic #11: 0
Num docs belonging to topic #12: 0
Num docs belonging to topic #13: 0
Num docs belonging to topic #14: 0
Num docs belonging to topic #15: 0
Num docs belonging to topic #16: 0
Num docs belonging to topic #17: 0
Num docs belonging to topic #18: 0
Num docs belonging to topic #19: 0
Num docs belonging to topic #20: 0
Num docs belonging to topic #21: 0
Num docs belonging to topic #22: 0


100%|██████████| 23/23 [00:00<?, ?it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\4137851866.py:91: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_40760\4137851866.py:92: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer1.save()


In [ ]:
full_docs = pd.read_excel('0527_full_content_cc.xlsx',dtype=object)
full_docs = full_docs.drop(['Unnamed: 0'], axis=1)
full_docs2 = full_docs[['No','corpus','Article Title','Abstract','Year']]

docs_zip = pd.ExcelWriter('sub/0527_final_topic_index_cc.xlsx', engine='xlsxwriter')

for t in sub['Topics']:

    topic_docs = pd.read_excel('sub/0525_results_plan_sub_topic_doc.xlsx', sheet_name='Topic_{}'.format(t))
    # print(t, len(topic_docs['mdl.docs']))
    df = full_docs2.merge(topic_docs, how = 'right', left_on = 'corpus', right_on = 'mdl.docs')
    # print(t, len(df))
    df2 = df.drop_duplicates(['Abstract'],keep='first',ignore_index=True)
    df2 = df2.dropna()
    df2['count'] = 1
    # print(t, df2['count'].sum())
    df2['doc_count'] = df2.groupby(['topic'])['count'].transform(sum)
    df2 = df2.sort_values('doc_count', ascending=False)
    content = df2.dropna()
    content = content[content['doc_count']>=10]
    content = content.reset_index(drop=True)
    content = content.drop(['No', 'mdl.docs'], axis=1)

    content.to_excel(docs_zip, sheet_name = 'Topic_{}'.format(t), index=False)

docs_zip.save()

FileNotFoundError: [Errno 2] No such file or directory: '0525_full_content_cc.xlsx'

In [ ]:
custom_tokenizer = CustomTokenizer()
sentence = [[] for _ in range(len(sub['Topics']))]
for r in sub['Topics']:
    content = pd.read_excel('Sub/0527_final_topic_index_cc.xlsx', sheet_name='Topic_{}'.format(r))

    sub_topics = list(content['topic'].unique())
    print(sub_topics)

    for i in sub_topics:
        preprocessing = []
        for line in tqdm(range(len(content.Abstract.values[content['topic'] == i]))):
            preprocessing.append(custom_tokenizer(content.Abstract.values[content['topic'] == i][line]))
        sentence[r].append(preprocessing)

[9, 7, 1, 5]


100%|██████████| 14/14 [00:00<00:00, 96.55it/s]


[0, 1, 8, 6]


100%|██████████| 23/23 [00:00<00:00, 99.30it/s] 


[3, 0, 6, 2, 5]


...

100%|██████████| 39/39 [00:00<00:00, 79.85it/s]


[3]


100%|██████████| 30/30 [00:00<00:00, 118.99it/s]


[0]


100%|██████████| 25/25 [00:00<00:00, 73.31it/s]


In [ ]:
len(sentence[0][1])

1534

In [ ]:
for r in sub['Topics']:

    writer = pd.ExcelWriter('sub/sub/0527_num_10_cc_sub_Topic{}.xlsx'.format(r), engine='openpyxl')
    content = pd.read_excel('Sub/0527_final_topic_index_cc.xlsx', sheet_name='Topic_{}'.format(r))
    sub_topics = list(content['topic'].unique())

    sub_sub = {'Topics' : [], 'Sub_topics' : [], 'Coherence_score' : []}

    for l in range(len(sub_topics)):

        corpus = tp.utils.Corpus()
        for line in sentence[r][l]:
            tokens = word_tokenize(line)
            corpus.add_doc(words = tokens)

        model_results = {'Topics': [], 'Coherence': [], 'Perplexity' : []}

        for j in range(2,11):

            mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=j, corpus=corpus, seed=99999)

            # Initiate MCMC burn-in
            mdl.burn_in = 100
            mdl.train(0)

            print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
            print('Removed top words:', mdl.removed_top_words)
            print('Training...', file=sys.stderr, flush=True)

            # Train model
            mcmc_iter = 1000
            quiet = False

            step=round(mcmc_iter*0.01)
            for i in range(0, mcmc_iter, step):
                mdl.train(step, workers=1)
                if not quiet:
                    ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

            coh = tp.coherence.Coherence(mdl, coherence='c_v')
            average_coherence = coh.get_score()
            topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

            print('Topic {}_Opti_topic {}_Coherence: {:.4}_Perplexity: {:.4}'.format(j, topic_len, average_coherence, mdl.perplexity))
            model_results['Topics'].append(topic_len)
            model_results['Coherence'].append(average_coherence)
            model_results['Perplexity'].append(mdl.perplexity)

        print('Topics : {}, Sub_topics : {}, Coherence_score : {:.4}'.format(sub_topics[l], model_results['Coherence'].index(max(model_results['Coherence']))+2, max(model_results['Coherence'])))
        sub_sub['Topics'].append(sub_topics[l])
        sub_sub['Sub_topics'].append(model_results['Coherence'].index(max(model_results['Coherence']))+2)
        sub_sub['Coherence_score'].append(max(model_results['Coherence']))

    pd.DataFrame(sub_sub).to_excel(writer, index=False)
    writer.save()


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2884310502.py:24: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


Num docs: 1555 , Vocab size: 1497 , Num words: 104232
Removed top words: ['policy', 'climate-change', 'adaptation', 'climate', 'urban', 'energy', 'local', 'city', 'action', 'process']


Training...


Topic 2_Opti_topic 2_Coherence: 0.3704_Perplexity: 1.137e+03
Num docs: 1555 , Vocab size: 1497 , Num words: 104232
Removed top words: ['policy', 'climate-change', 'adaptation', 'climate', 'urban', 'energy', 'local', 'city', 'action', 'process']


Training...


...

Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 2_Opti_topic 2_Coherence: 0.4744_Perplexity: 7.015
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 3_Opti_topic 3_Coherence: 0.4744_Perplexity: 7.489
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 4_Opti_topic 4_Coherence: 0.4744_Perplexity: 7.669
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 5_Opti_topic 1_Coherence: 0.4744_Perplexity: 6.428
Num docs: 25 , Vocab siz

Training...
Training...
Training...
Training...


Topic 6_Opti_topic 3_Coherence: 0.4744_Perplexity: 7.996
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 7_Opti_topic 5_Coherence: 0.4744_Perplexity: 8.082
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 8_Opti_topic 8_Coherence: 0.4744_Perplexity: 13.29
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
Topic 9_Opti_topic 3_Coherence: 0.4744_Perplexity: 8.289
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2884310502.py:55: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Topic 10_Opti_topic 6_Coherence: 0.4744_Perplexity: 8.438
Topics : 0, Sub_topics : 5, Coherence_score : 0.4744


In [ ]:
for t in sub['Topics']:

    sub_sub = pd.read_excel('sub/sub/0527_num_10_cc_sub_Topic{}.xlsx'.format(t),dtype=object)
    writer = pd.ExcelWriter('sub/sub/result/0527_results_cc_sub_topic{}.xlsx'.format(t), engine='xlsxwriter')

    for l in range(len(sub_sub['Topics'])):

        corpus = tp.utils.Corpus()
        for line in sentence[t][l]:
            tokens = word_tokenize(line)
            corpus.add_doc(words = tokens)

        print(sub_sub['Topics'][l], sub_sub['Sub_topics'][l]+2, sub_sub['Coherence_score'][l])

        mdl = tp.HDPModel(tw=tp.TermWeight.IDF, min_cf=5, min_df=10, rm_top=10, initial_k=sub_sub['Sub_topics'][l]+2, corpus=corpus, seed=99999)

        # Initiate MCMC burn-in
        mdl.burn_in = 100
        mdl.train(0)

        print('Num docs:', len(mdl.docs), ', Vocab size:', mdl.num_vocabs, ', Num words:', mdl.num_words)
        print('Removed top words:', mdl.removed_top_words)
        print('Training...', file=sys.stderr, flush=True)

        # Train model
        mcmc_iter = 1000
        quiet = False

        step=round(mcmc_iter*0.01)
        for i in range(0, mcmc_iter, step):
            mdl.train(step, workers=1)
            if not quiet:
                ('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, mdl.ll_per_word, mdl.live_k))

        coh = tp.coherence.Coherence(mdl, coherence='c_v')
        average_coherence = coh.get_score()
        topic_len = np.array(np.nonzero(mdl.get_count_by_topics())).size

        print('topic{}_Coherence: {:.4}_Perplexity: {:.4}'.format(topic_len, average_coherence, mdl.perplexity))

        topic_results = {'Topics': [], 'document_count': [], 'represent': [], 'words': [], 'word_probability': []}

        important_topics = [k for k, v in sorted(enumerate(mdl.get_count_by_topics()), key=lambda x:x[1], reverse=True)]
        important_topics.sort()

        # 먼저 토픽 개수만큼 공간을 만들어줍니다.
        buckets = [[] for _ in range(len(important_topics))]

        # extract candidates for auto topic labeling
        extractor = tp.label.PMIExtractor(min_cf=5, min_df=10, min_len=1, max_len=4, max_cand=10000)
        cands = extractor.extract(mdl)

        labeler = tp.label.FoRelevance(mdl, cands, min_df=10, smoothing=1e-2, mu=0.25)

        topic_doc = {'mdl.docs': [],'topic': []}

        for k in tqdm(important_topics):
            if not mdl.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)

            # 전체 문헌에 대해, top 1 주제 번호를 찾아 그 그룹에 해당 문헌을 포함시킵니다.
            for d in mdl.docs:
                topic = d.get_topics(top_n=1)[0][0]
                if k == topic:
                    buckets[k].append(topic)

            topic_results['Topics'].append(k)
            topic_results['document_count'].append(len(buckets[k]))
            label = "Labels :", ', '.join(label for label, score in labeler.get_topic_labels(k, top_n=40))
            topic_results['represent'].append(label)
            words = "Words :", ', '.join(word for word, prob in mdl.get_topic_words(k, top_n=40))
            word = [word for word, prob in mdl.get_topic_words(k, top_n=40)]
            prob = [np.round(prob,6) for word, prob in mdl.get_topic_words(k, top_n=40)]
            topic_results['words'].append(words)
            word_porobability = dict(zip(word, prob))
            topic_results['word_probability'].append(word_porobability)

        pd.DataFrame(topic_results).to_excel(writer, sheet_name = 'Topic_{}'.format(l), index=False)

    writer.save()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:19: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


9 6 0.5815222464501858
Num docs: 1555 , Vocab size: 1497 , Num words: 104232
Removed top words: ['policy', 'climate-change', 'adaptation', 'climate', 'urban', 'energy', 'local', 'city', 'action', 'process']
topic6_Coherence: 0.5815_Perplexity: 1.087e+03


100%|██████████| 7/7 [00:00<00:00, 388.89it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:19: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


7 8 0.6122079645097254
Num docs: 1534 , Vocab size: 1542 , Num words: 112353
Removed top words: ['climate-change', 'adaptation', 'policy', 'urban', 'risk', 'water', 'management', 'strategy', 'climate', 'city']
topic8_Coherence: 0.6122_Perplexity: 1.136e+03


100%|██████████| 10/10 [00:00<00:00, 416.43it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:19: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


...

100%|██████████| 33/33 [00:00<00:00, 8249.61it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:79: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:19: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


3 5 0.2477004866458868
Num docs: 30 , Vocab size: 10 , Num words: 196
Removed top words: ['design', 'material', 'green', 'low-carbon', 'environmental', 'environment', 'protection', 'concept', 'energy', 'development']
topic1_Coherence: 0.2477_Perplexity: 12.75


100%|██████████| 19/19 [00:00<?, ?it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:79: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:19: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(0)
Training...


0 5 0.4743825529302869
Num docs: 25 , Vocab size: 5 , Num words: 115
Removed top words: ['specie', 'conservation', 'area', 'climate-change', 'approach', 'adaptation', 'network', 'future', 'climate', 'habitat']
topic1_Coherence: 0.4744_Perplexity: 6.428


100%|██████████| 7/7 [00:00<00:00, 7010.54it/s]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_59236\2106772570.py:79: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
